In [1]:
import pandas as pd
import pickle
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_sample_weight
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import os
import pickle
import numpy as np
from tqdm import tqdm

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
def evaluate(prediction_rate,X,Y,xgb_model_loaded,junction):
    history = [0]  # Assuming no incident at start
    incident_counter = 0
    detected_counter = 0
    new_incident_flag = False
    incident_detected_flag = False
    predictions_count = 0
    i = 0  # Time since last new incident
    false_alarm_counter = 0
    detection_times = []  # To calculate mean time for detection
    counter = 1
    for row,yrow in tqdm(zip(X,Y)):
        
        if counter% prediction_rate == 0:
            
            predictions_count+=1
            predicted = xgb_model_loaded.predict([row])[0] 
            # print(predicted)
            
            if len(history) > 0:
                previous_label = history[-1]
            else:
                previous_label = 0  # Default to no incident for the first element

            # Update history
            current_label = 1 if yrow else 0
            history.append(current_label)

            # New incident started
            if previous_label == 0 and current_label == 1:
                incident_counter += 1
                new_incident_flag = True
                incident_detected_flag = False  # Reset detection flag for new incident
                i = 0  # Reset timer for new incident
                # print("New incident started")

            # Incident detected
            if new_incident_flag and predicted == 1 and not incident_detected_flag:
                detected_counter += 1
                detection_times.append(i)
                incident_detected_flag = True
                # print(f"Incident Detected by model after {i} timesteps.")

            # Incident finished
            if previous_label == 1 and current_label == 0:
                new_incident_flag = False
                # print("Incident Finished")


            # False alarm raised
            if not new_incident_flag and predicted == 1 and not (1 in history[-1000:]):
                false_alarm_counter += 1
                
        # Increment timer if incident ongoing but not yet detected
        if new_incident_flag and not incident_detected_flag:
            i += 1
        counter+=1
        
    false_alarm_rate = 100*false_alarm_counter/predictions_count
    arr = np.array(detection_times)
    row = {
             "junctions" : junction,
            "prediction_rate" : prediction_rate, 
           "incident_counter": incident_counter,
           "detected_counter":detected_counter,
           "mttd":arr.mean(),
           "false_alarm_counter":false_alarm_counter,
           "false_alarm_rate": false_alarm_rate,
           "predictions_count":predictions_count
        
    }    
    return row

In [5]:
df = pd.read_csv("/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/xgboost/evaluation_proccessed/proc_8_600_600.csv")

/tmp/ipykernel_2789116/1049740314.py:1: DtypeWarning: Columns (129,131,132,134) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/xgboost/evaluation_proccessed/proc_8_600_600.csv")


In [6]:
road_name_edge_id = {
1:{"934465920","5614812#0","889439250","436794672#0","1078715158"},
2:{"532215357#0","436794668#0","436794668#7","436794677#0","436794673#0"},
3:{"436791116","436791119#0","436791122#0","436791121#0","436791111"},
4:{"1070423862#0","5602753#1","5602753#2","436790493#0","533573789#0","533573789#2","436790492#0","436790484","512811687#0"},
5:{"436942365#0","436942382#0","436942369#0","436942367","436942384#0","436942364","436942372","395215600","966303717","436942386#0"},
6:{"532227836","532227834#0","436789544#0","436789576#0","436789539#0","436789539#2","436789539#7","436789570#0"},
7:{"-436942361#7","-436942361#3","-436942361#1","-436942358#5","-436942358#3","-436942358#0","345713658#0"},
8:{"-436789319","395490730","1051025192"},
9:{"436940273#0","692089619#0","692089616#0","692089616#2","436940272#0","436940271#0","5635238","692089613#0","692089613#2","692089613#6","692089611#0"},
10:{"512810351#0","436943782","436943780","436943781"},
11:{"436943721","436943716","436943727#0","406379830#0","436943736","436943731#0","436943728","436943723","436943726","436943720","436943747#0","436943754#0","436943740","436943735","436943729","436943741#0","436943752#0"},
12:{"-436943756#2","-911576955#2","909831620","911576960","-327757100#6"}

}

In [7]:
df.head()

,Unnamed: 0,step,rolling_junction_mean_speed_0,rolling_traffic_count_0,rolling_traffic_occupancy_0,rolling_junction_mean_speed_1,rolling_traffic_count_1,rolling_traffic_occupancy_1,rolling_junction_mean_speed_2,rolling_traffic_count_2,rolling_traffic_occupancy_2,rolling_junction_mean_speed_3,rolling_traffic_count_3,rolling_traffic_occupancy_3,rolling_junction_mean_speed_4,rolling_traffic_count_4,rolling_traffic_occupancy_4,rolling_junction_mean_speed_5,rolling_traffic_count_5,rolling_traffic_occupancy_5,rolling_junction_mean_speed_6,rolling_traffic_count_6,rolling_traffic_occupancy_6,rolling_junction_mean_speed_7,rolling_traffic_count_7,rolling_traffic_occupancy_7,rolling_junction_mean_speed_8,rolling_traffic_count_8,rolling_traffic_occupancy_8,rolling_junction_mean_speed_9,rolling_traffic_count_9,rolling_traffic_occupancy_9,rolling_junction_mean_speed_10,rolling_traffic_count_10,rolling_traffic_occupancy_10,rolling_junction_mean_speed_11,rolling_traffic_count_11,rolling_traffic_occupancy_11,rolling_junction_mean_speed_12,rolling_traffic_count_12,rolling_traffic_occupancy_12,rolling_junction_mean_speed_13,rolling_traffic_count_13,rolling_traffic_occupancy_13,rolling_junction_mean_speed_14,rolling_traffic_count_14,rolling_traffic_occupancy_14,rolling_junction_mean_speed_15,rolling_traffic_count_15,rolling_traffic_occupancy_15,rolling_junction_mean_speed_16,rolling_traffic_count_16,rolling_traffic_occupancy_16,rolling_junction_mean_speed_17,rolling_traffic_count_17,rolling_traffic_occupancy_17,rolling_junction_mean_speed_18,rolling_traffic_count_18,rolling_traffic_occupancy_18,rolling_junction_mean_speed_19,rolling_traffic_count_19,rolling_traffic_occupancy_19,rolling_junction_mean_speed_20,rolling_traffic_count_20,rolling_traffic_occupancy_20,rolling_junction_mean_speed_21,rolling_traffic_count_21,rolling_traffic_occupancy_21,rolling_junction_mean_speed_22,rolling_traffic_count_22,rolling_traffic_occupancy_22,rolling_junction_mean_speed_23,rolling_traffic_count_23,rolling_traffic_occupancy_23,rolling_junction_mean_speed_24,rolling_traffic_count_24,rolling_traffic_occupancy_24,rolling_junction_mean_speed_25,rolling_traffic_count_25,rolling_traffic_occupancy_25,rolling_junction_mean_speed_26,rolling_traffic_count_26,rolling_traffic_occupancy_26,rolling_junction_mean_speed_27,rolling_traffic_count_27,rolling_traffic_occupancy_27,rolling_junction_mean_speed_28,rolling_traffic_count_28,rolling_traffic_occupancy_28,rolling_junction_mean_speed_29,rolling_traffic_count_29,rolling_traffic_occupancy_29,rolling_junction_mean_speed_30,rolling_traffic_count_30,rolling_traffic_occupancy_30,rolling_junction_mean_speed_31,rolling_traffic_count_31,rolling_traffic_occupancy_31,rolling_junction_mean_speed_32,rolling_traffic_count_32,rolling_traffic_occupancy_32,rolling_junction_mean_speed_33,rolling_traffic_count_33,rolling_traffic_occupancy_33,rolling_junction_mean_speed_34,rolling_traffic_count_34,rolling_traffic_occupancy_34,rolling_junction_mean_speed_35,rolling_traffic_count_35,rolling_traffic_occupancy_35,rolling_junction_mean_speed_36,rolling_traffic_count_36,rolling_traffic_occupancy_36,rolling_junction_mean_speed_37,rolling_traffic_count_37,rolling_traffic_occupancy_37,rolling_junction_mean_speed_38,rolling_traffic_count_38,rolling_traffic_occupancy_38,rolling_junction_mean_speed_39,rolling_traffic_count_39,rolling_traffic_occupancy_39,rolling_junction_mean_speed_40,rolling_traffic_count_40,rolling_traffic_occupancy_40,rolling_junction_mean_speed_41,rolling_traffic_count_41,rolling_traffic_occupancy_41,time_of_day,incident_edge,incident_start_time,incident_type,accident_id,accident_duration,incident_lane,accident_label,rolling_travel_time_3_5,rolling_travel_time_10_4,rolling_travel_time_8_11,rolling_travel_time_14_9,rolling_travel_time_30_33,rolling_travel_time_38_31,rolling_travel_time_34_39,rolling_travel_time_42_35,rolling_travel_time_32_2,rolling_travel_time_1_29,rolling_travel_time_26_13,rolling_travel_time_12_27,rolling_travel_time

In [8]:
df =df .drop(["Unnamed: 0", "step"],axis=1)

In [9]:
df.shape

(1296000, 158)

In [10]:
def standardize(col):
   return (col - col.mean()) / col.std()


In [11]:
# df = df[df["accident_label"] == True]
# df['label'] = None
# df =df.fillna(-1)

# for category, edges in road_name_edge_id.items():
# # Find rows where incident_edge is in the current set of edges
#     mask = df['incident_edge'].isin(edges)
# # Set the appropriate label column to 1 for these rows
#     df.loc[mask, 'label'] = category-1  # Adjusted index by -1 for zero-based indexing

# Y = df["label"]


    
# X = df.drop(["incident_edge","incident_start_time","incident_type","accident_id","accident_duration","incident_lane","accident_label","label"],axis=1)


In [ ]:
# Severity

In [19]:
df.head()

,rolling_junction_mean_speed_0,rolling_traffic_count_0,rolling_traffic_occupancy_0,rolling_junction_mean_speed_1,rolling_traffic_count_1,rolling_traffic_occupancy_1,rolling_junction_mean_speed_2,rolling_traffic_count_2,rolling_traffic_occupancy_2,rolling_junction_mean_speed_3,rolling_traffic_count_3,rolling_traffic_occupancy_3,rolling_junction_mean_speed_4,rolling_traffic_count_4,rolling_traffic_occupancy_4,rolling_junction_mean_speed_5,rolling_traffic_count_5,rolling_traffic_occupancy_5,rolling_junction_mean_speed_6,rolling_traffic_count_6,rolling_traffic_occupancy_6,rolling_junction_mean_speed_7,rolling_traffic_count_7,rolling_traffic_occupancy_7,rolling_junction_mean_speed_8,rolling_traffic_count_8,rolling_traffic_occupancy_8,rolling_junction_mean_speed_9,rolling_traffic_count_9,rolling_traffic_occupancy_9,rolling_junction_mean_speed_10,rolling_traffic_count_10,rolling_traffic_occupancy_10,rolling_junction_mean_speed_11,rolling_traffic_count_11,rolling_traffic_occupancy_11,rolling_junction_mean_speed_12,rolling_traffic_count_12,rolling_traffic_occupancy_12,rolling_junction_mean_speed_13,rolling_traffic_count_13,rolling_traffic_occupancy_13,rolling_junction_mean_speed_14,rolling_traffic_count_14,rolling_traffic_occupancy_14,rolling_junction_mean_speed_15,rolling_traffic_count_15,rolling_traffic_occupancy_15,rolling_junction_mean_speed_16,rolling_traffic_count_16,rolling_traffic_occupancy_16,rolling_junction_mean_speed_17,rolling_traffic_count_17,rolling_traffic_occupancy_17,rolling_junction_mean_speed_18,rolling_traffic_count_18,rolling_traffic_occupancy_18,rolling_junction_mean_speed_19,rolling_traffic_count_19,rolling_traffic_occupancy_19,rolling_junction_mean_speed_20,rolling_traffic_count_20,rolling_traffic_occupancy_20,rolling_junction_mean_speed_21,rolling_traffic_count_21,rolling_traffic_occupancy_21,rolling_junction_mean_speed_22,rolling_traffic_count_22,rolling_traffic_occupancy_22,rolling_junction_mean_speed_23,rolling_traffic_count_23,rolling_traffic_occupancy_23,rolling_junction_mean_speed_24,rolling_traffic_count_24,rolling_traffic_occupancy_24,rolling_junction_mean_speed_25,rolling_traffic_count_25,rolling_traffic_occupancy_25,rolling_junction_mean_speed_26,rolling_traffic_count_26,rolling_traffic_occupancy_26,rolling_junction_mean_speed_27,rolling_traffic_count_27,rolling_traffic_occupancy_27,rolling_junction_mean_speed_28,rolling_traffic_count_28,rolling_traffic_occupancy_28,rolling_junction_mean_speed_29,rolling_traffic_count_29,rolling_traffic_occupancy_29,rolling_junction_mean_speed_30,rolling_traffic_count_30,rolling_traffic_occupancy_30,rolling_junction_mean_speed_31,rolling_traffic_count_31,rolling_traffic_occupancy_31,rolling_junction_mean_speed_32,rolling_traffic_count_32,rolling_traffic_occupancy_32,rolling_junction_mean_speed_33,rolling_traffic_count_33,rolling_traffic_occupancy_33,rolling_junction_mean_speed_34,rolling_traffic_count_34,rolling_traffic_occupancy_34,rolling_junction_mean_speed_35,rolling_traffic_count_35,rolling_traffic_occupancy_35,rolling_junction_mean_speed_36,rolling_traffic_count_36,rolling_traffic_occupancy_36,rolling_junction_mean_speed_37,rolling_traffic_count_37,rolling_traffic_occupancy_37,rolling_junction_mean_speed_38,rolling_traffic_count_38,rolling_traffic_occupancy_38,rolling_junction_mean_speed_39,rolling_traffic_count_39,rolling_traffic_occupancy_39,rolling_junction_mean_speed_40,rolling_traffic_count_40,rolling_traffic_occupancy_40,rolling_junction_mean_speed_41,rolling_traffic_count_41,rolling_traffic_occupancy_41,time_of_day,incident_edge,incident_start_time,incident_type,accident_id,accident_duration,incident_lane,accident_label,rolling_travel_time_3_5,rolling_travel_time_10_4,rolling_travel_time_8_11,rolling_travel_time_14_9,rolling_travel_time_30_33,rolling_travel_time_38_31,rolling_travel_time_34_39,rolling_travel_time_42_35,rolling_travel_time_32_2,rolling_travel_time_1_29,rolling_travel_time_26_13,rolling_travel_time_12_27,rolling_travel_time_40_25,rolling_t

In [20]:
df2 = df.iloc[::30]

In [21]:
df2.head()

,rolling_junction_mean_speed_0,rolling_traffic_count_0,rolling_traffic_occupancy_0,rolling_junction_mean_speed_1,rolling_traffic_count_1,rolling_traffic_occupancy_1,rolling_junction_mean_speed_2,rolling_traffic_count_2,rolling_traffic_occupancy_2,rolling_junction_mean_speed_3,rolling_traffic_count_3,rolling_traffic_occupancy_3,rolling_junction_mean_speed_4,rolling_traffic_count_4,rolling_traffic_occupancy_4,rolling_junction_mean_speed_5,rolling_traffic_count_5,rolling_traffic_occupancy_5,rolling_junction_mean_speed_6,rolling_traffic_count_6,rolling_traffic_occupancy_6,rolling_junction_mean_speed_7,rolling_traffic_count_7,rolling_traffic_occupancy_7,rolling_junction_mean_speed_8,rolling_traffic_count_8,rolling_traffic_occupancy_8,rolling_junction_mean_speed_9,rolling_traffic_count_9,rolling_traffic_occupancy_9,rolling_junction_mean_speed_10,rolling_traffic_count_10,rolling_traffic_occupancy_10,rolling_junction_mean_speed_11,rolling_traffic_count_11,rolling_traffic_occupancy_11,rolling_junction_mean_speed_12,rolling_traffic_count_12,rolling_traffic_occupancy_12,rolling_junction_mean_speed_13,rolling_traffic_count_13,rolling_traffic_occupancy_13,rolling_junction_mean_speed_14,rolling_traffic_count_14,rolling_traffic_occupancy_14,rolling_junction_mean_speed_15,rolling_traffic_count_15,rolling_traffic_occupancy_15,rolling_junction_mean_speed_16,rolling_traffic_count_16,rolling_traffic_occupancy_16,rolling_junction_mean_speed_17,rolling_traffic_count_17,rolling_traffic_occupancy_17,rolling_junction_mean_speed_18,rolling_traffic_count_18,rolling_traffic_occupancy_18,rolling_junction_mean_speed_19,rolling_traffic_count_19,rolling_traffic_occupancy_19,rolling_junction_mean_speed_20,rolling_traffic_count_20,rolling_traffic_occupancy_20,rolling_junction_mean_speed_21,rolling_traffic_count_21,rolling_traffic_occupancy_21,rolling_junction_mean_speed_22,rolling_traffic_count_22,rolling_traffic_occupancy_22,rolling_junction_mean_speed_23,rolling_traffic_count_23,rolling_traffic_occupancy_23,rolling_junction_mean_speed_24,rolling_traffic_count_24,rolling_traffic_occupancy_24,rolling_junction_mean_speed_25,rolling_traffic_count_25,rolling_traffic_occupancy_25,rolling_junction_mean_speed_26,rolling_traffic_count_26,rolling_traffic_occupancy_26,rolling_junction_mean_speed_27,rolling_traffic_count_27,rolling_traffic_occupancy_27,rolling_junction_mean_speed_28,rolling_traffic_count_28,rolling_traffic_occupancy_28,rolling_junction_mean_speed_29,rolling_traffic_count_29,rolling_traffic_occupancy_29,rolling_junction_mean_speed_30,rolling_traffic_count_30,rolling_traffic_occupancy_30,rolling_junction_mean_speed_31,rolling_traffic_count_31,rolling_traffic_occupancy_31,rolling_junction_mean_speed_32,rolling_traffic_count_32,rolling_traffic_occupancy_32,rolling_junction_mean_speed_33,rolling_traffic_count_33,rolling_traffic_occupancy_33,rolling_junction_mean_speed_34,rolling_traffic_count_34,rolling_traffic_occupancy_34,rolling_junction_mean_speed_35,rolling_traffic_count_35,rolling_traffic_occupancy_35,rolling_junction_mean_speed_36,rolling_traffic_count_36,rolling_traffic_occupancy_36,rolling_junction_mean_speed_37,rolling_traffic_count_37,rolling_traffic_occupancy_37,rolling_junction_mean_speed_38,rolling_traffic_count_38,rolling_traffic_occupancy_38,rolling_junction_mean_speed_39,rolling_traffic_count_39,rolling_traffic_occupancy_39,rolling_junction_mean_speed_40,rolling_traffic_count_40,rolling_traffic_occupancy_40,rolling_junction_mean_speed_41,rolling_traffic_count_41,rolling_traffic_occupancy_41,time_of_day,incident_edge,incident_start_time,incident_type,accident_id,accident_duration,incident_lane,accident_label,rolling_travel_time_3_5,rolling_travel_time_10_4,rolling_travel_time_8_11,rolling_travel_time_14_9,rolling_travel_time_30_33,rolling_travel_time_38_31,rolling_travel_time_34_39,rolling_travel_time_42_35,rolling_travel_time_32_2,rolling_travel_time_1_29,rolling_travel_time_26_13,rolling_travel_time_12_27,rolling_travel_time_40_25,rolling_t

In [22]:
# Incident Detection
# df = df[600:]
df = df2
Y = df["accident_label"]
X = df.drop(["incident_edge","incident_start_time","incident_type","accident_id","accident_duration","incident_lane","accident_label"],axis=1)

In [23]:
X = X.apply(standardize)




In [24]:
X= X.values
Y= Y.values

In [25]:
model_path = "/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/xgboost/experiment_saved_models/8jun_inci_detect/xg_model.pkl"
xgb_model_loaded = pickle.load(open(model_path, "rb"))

In [26]:
# Direct Evaluation - but in reality we run prediction every 1 minute and when an incident flag is raised and detected we have a cooling period after incident clearance
y_pred = xgb_model_loaded.predict(X)
y_pred = list(y_pred)
y_test = list(Y)

accuracy = accuracy_score(y_test, y_pred)*100
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy))
print(f"FAR {100-accuracy}")
print("precision: %.2f%%" % (precision * 100.0))
print("f1: %.2f%%" % (f1 * 100.0))
print("recall: %.2f%%" % (recall * 100.0))

Accuracy: 63.25%
FAR 36.74849467345993
precision: 40.36%
f1: 53.15%
recall: 77.83%


In [17]:
junction = 8
print(evaluate(30,X,Y,xgb_model_loaded,junction))

1295400it [00:12, 107615.93it/s]

{'junctions': 8, 'prediction_rate': 30, 'incident_counter': 100, 'detected_counter': 100, 'mttd': 96.9, 'false_alarm_counter': 4772, 'false_alarm_rate': 11.051412691060676, 'predictions_count': 43180}
